# Data Preparation


In [16]:
# --------------------------------------------------------------------------------------------------------
# ciaFactBook.ipynb -- data preparation, inspection, and visualization 
# --------------------------------------------------------------------------------------------------------
# 0.01|03.01.22|AB|creation
# --------------------------------------------------------------------------------------------------------

# initialize

import numpy as np
import pandas as pd
#import glob
#import os
import matplotlib.pyplot as plt

#from datetime import datetime, date 
import re # regular expression
#import warnings
import pickle
import seaborn as sns

# %matplotlib inline
# %matplotlib notebook

PATH_DATA = r'../../data/'
INTERACTIVE = False
VALUE_COLNAME = 'language'  # name for new column containing value

if INTERACTIVE:
    %matplotlib notebook
else:
    %matplotlib inline
    

## Languages per country (official, minority)

_The number of persons having their usual residence in a country on 1 January of the respective year. When usually resident population is not available, countries may report legal or registered residents._

https://www.cia.gov/the-world-factbook/about/archives/


In [2]:
fileName = PATH_DATA + 'countries_languages_all.csv'
# header = 1st line
data = pd.read_csv(fileName, sep=',', header=0) 

In [3]:
data

,Country,Recognized Languages
0,Afghanistan,Afghan Persian or Dari (official) 77% (Dari fu...
1,Albania,Albanian 98.8% (official - derived from Tosk d...
2,Algeria,"Arabic (official), French (lingua franca), Ber..."
3,Andorra,"Catalan (official), French, Castilian, Portuguese"
4,Angola,"Portuguese 71.2% (official), Umbundu 23%, Kiko..."
...,...,...
209,Vietnam,"Vietnamese (official), English (increasingly f..."
210,Western Sahara (proposed state),"Standard Arabic, Hassaniya Arabic, Moroccan Ar..."
211,Yemen,Arabic (official)
212,Zambia,"Bemba 33.4%, Nyanja 14.7%, Tonga 11.4%, Lozi 5..."


TBA:  - map Country -> country code
      - select relevant countries
      
Second (unstructured) column is transformed as follows:
- Split text into 1 column per language
- add one column for share / official(yes/no)

In [4]:
# data['Recognized Languages'].str.extract([A-Z][a-z]+ )

# https://stackoverflow.com/questions/14745022/how-to-split-a-dataframe-string-column-into-two-columns/21296915#21296915
#data['Recognized Languages'].str.split(',', expand=True)

data = data.join(data['Recognized Languages'].str.split(',', expand=True))

In [5]:
# drop split column
data.pop('Recognized Languages')
data

,Country,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Afghanistan,Afghan Persian or Dari (official) 77% (Dari fu...,Pashto (official) 48%,Uzbek 11%,English 6%,Turkmen 3%,Urdu 3%,Pashayi 1%,Nuristani 1%,Arabic 1%,Balochi 1% (2017 est.),None,None,None,None,None,None,None,None,None
1,Albania,Albanian 98.8% (official - derived from Tosk d...,Greek 0.5%,other 0.6% (including Macedonian,Romani,Vlach,Turkish,Italian,and Serbo-Croatian),unspecified 0.1% (2011 est.),None,None,None,None,None,None,None,None,None,None
2,Algeria,Arabic (official),French (lingua franca),Berber or Tamazight (official); dialects incl...,Shawiya Berber (Tacawit),Mzab Berber,Tuareg Berber (Tamahaq),None,None,None,None,None,None,None,None,None,None,None,None,None
3,Andorra,Catalan (official),French,Castilian,Portuguese,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Angola,Portuguese 71.2% (official),Umbundu 23%,Kikongo 8.2%,Kimbundu 7.8%,Chokwe 6.5%,Nhaneca 3.4%,Nganguela 3.1%,Fiote 2.4%,Kwanhama 2.3%,Muhumbi 2.1%,Luvale 1%,other 3.6% (2014 est.),None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Vietnam,Vietnamese (official),English (increasingly favored as a second lan...,some French,Chinese,and Khmer,mountain area languages (Mon-Khmer and Malayo...,None,None,None,None,None,None,None,None,None,None,None,None,None
210,Western Sahara (proposed state),Standard Arabic,Hassaniya Arabic,Moroccan Arabic,Berber,Spanish,French,None,None,None,None,None,None,None,None,None,None,None,None,None
211,Yemen,Arabic (official),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
212,Zambia,Bemba 33.4%,Nyanja 14.7%,Tonga 11.4%,Lozi 5.5%,Chewa 4.5%,Nsenga 2.9%,Tumbuka 2.5%,Lunda (North Western) 1.9%,Kaonde 1.8%,Lala 1.8%,Lamba 1.8%,English (official) 1.7%,Luvale 1.5%,Mambwe 1.3%,Namwanga 1.2%,Lenje 1.1%,Bisa 1%,other 9.7%,unspecified 0.2% (2010 est.)


In [6]:
# create a row for each country-language combination (pivot wide -> long)
# key is country (for each language and country, a row is to be created)
#data = 
data = data.melt(['Country'], value_name = 'languageRaw', var_name = 'langId')
#, var_name = 'langId', value_vars = None, value_name = None)   #, value_name = VALUE_COLNAME)


In [7]:
data

,Country,langId,languageRaw
0,Afghanistan,0,Afghan Persian or Dari (official) 77% (Dari fu...
1,Albania,0,Albanian 98.8% (official - derived from Tosk d...
2,Algeria,0,Arabic (official)
3,Andorra,0,Catalan (official)
4,Angola,0,Portuguese 71.2% (official)
...,...,...,...
4061,Vietnam,18,None
4062,Western Sahara (proposed state),18,None
4063,Yemen,18,None
4064,Zambia,18,unspecified 0.2% (2010 est.)


In [8]:
# remove empty languages
data = data.dropna()
data
# note: the number of languages spoken in a given country is equal to max(langId)+1; 
# most languages are spoken in Zambia (18)

,Country,langId,languageRaw
0,Afghanistan,0,Afghan Persian or Dari (official) 77% (Dari fu...
1,Albania,0,Albanian 98.8% (official - derived from Tosk d...
2,Algeria,0,Arabic (official)
3,Andorra,0,Catalan (official)
4,Angola,0,Portuguese 71.2% (official)
...,...,...,...
3572,Panama,16,Japanese)
3636,Zambia,16,Bisa 1%
3637,Zimbabwe,16,and Xhosa)
3850,Zambia,17,other 9.7%


In [9]:
# -- create an index
data.set_index(['Country', 'langId'], inplace = True)

In [10]:
# TBA: filter European countries / add country code
data


,,languageRaw
Country,langId,
Afghanistan,0,Afghan Persian or Dari (official) 77% (Dari fu...
Albania,0,Albanian 98.8% (official - derived from Tosk d...
Algeria,0,Arabic (official)
Andorra,0,Catalan (official)
Angola,0,Portuguese 71.2% (official)
...,...,...
Panama,16,Japanese)
Zambia,16,Bisa 1%
Zimbabwe,16,and Xhosa)


In [11]:
# mark official languages (new column official [Boolean])
pattern = "official"  
# data[data['languageRaw'].str.contains(pattern)]
data['official'] = data['languageRaw'].str.contains(pattern)


<ipython-input-11-3632ec7a8d7a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['official'] = data['languageRaw'].str.contains(pattern)


In [12]:
# data['languageRaw']
data

,,languageRaw,official
Country,langId,,
Afghanistan,0,Afghan Persian or Dari (official) 77% (Dari fu...,True
Albania,0,Albanian 98.8% (official - derived from Tosk d...,True
Algeria,0,Arabic (official),True
Andorra,0,Catalan (official),True
Angola,0,Portuguese 71.2% (official),True
...,...,...,...
Panama,16,Japanese),False
Zambia,16,Bisa 1%,False
Zimbabwe,16,and Xhosa),False


In [13]:
# Extract actual language name, identified as the first capitalized word (this heuristic introduces some NA's)
pattern = r'(?P<language>[A-Z][a-z]+)'  # named group becomes new column 'language'
data['languageRaw'].str.extract(pattern)
# data = data[data[colName].str.fullmatch(pattern)]

,,language
Country,langId,
Afghanistan,0,Afghan
Albania,0,Albanian
Algeria,0,Arabic
Andorra,0,Catalan
Angola,0,Portuguese
...,...,...
Panama,16,Japanese
Zambia,16,Bisa
Zimbabwe,16,Xhosa


In [14]:
data = data.join(data['languageRaw'].str.extract(pattern))
data = data.dropna()

In [15]:
data

,,languageRaw,official,language
Country,langId,,,
Afghanistan,0,Afghan Persian or Dari (official) 77% (Dari fu...,True,Afghan
Albania,0,Albanian 98.8% (official - derived from Tosk d...,True,Albanian
Algeria,0,Arabic (official),True,Arabic
Andorra,0,Catalan (official),True,Catalan
Angola,0,Portuguese 71.2% (official),True,Portuguese
...,...,...,...,...
Zambia,15,Lenje 1.1%,False,Lenje
Zimbabwe,15,Venda,False,Venda
Panama,16,Japanese),False,Japanese


In [19]:
# save processed data
fileName = PATH_DATA + 'countries_languages_all.pkl'
with open(fileName, 'wb') as outp:        
        pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)      